In [72]:
example = """
.#.
..#
###
""".strip().splitlines()

with open('day17.txt', 'r') as f:
    data = f.readlines()

## Strategy
We're using an immutable `State` construct which is transformed according to a set of rules. This allows us to snapshot the state for evaluation before we move to the next step.

We have the challenge of needing to introduce neighbouring cubes (which may start in an active state depending on the number of neighbours) so it is necessary to first build up
a set of the new state's cube space and then iterating to the new state.

## Cube

The Cube type implements the logic controlling the changes in cube state, as well as how it determines which cubes it considers neighbours. The rest of this class (`__eq__`, `__hash__` etc.) exist simply to make interop with Python a little bit easier.

In [196]:
from typing import Dict, Tuple, Iterator

class DimensionalScanner(object):
    def __init__(self, *dimensions: Tuple[Tuple[int, int]]):
        self.dimensions = dimensions

    def scan(self):
        scan_range = [1]
        options = [
            (max(*dimension) - min(*dimension) + 1)
            for dimension in self.dimensions
        ]

        for fi, dimension in enumerate(self.dimensions):
            scan_range.append(scan_range[fi] * options[fi])

        for i in range(scan_range[len(self.dimensions)]):
            yield tuple(
                (int(i/scan_range[fi]) % options[fi]) + dimension[0]
                for fi, dimension in enumerate(self.dimensions)
            )

test_scanner = DimensionalScanner((-1,1), (-1,1))
test_scanner_set = set(test_scanner.scan())
assert test_scanner_set == set([
    (-1, -1),
    (-1, 0),
    (-1, 1),
    (0, -1),
    (0, 0),
    (0, 1),
    (1, -1),
    (1, 0),
    (1, 1),
])

assert len(set(DimensionalScanner((-1, 1), (-1, 1), (-1, 1)).scan())) == 27
assert set(DimensionalScanner((-1, 1), (-1, 1), (-1, 1)).scan()) == set([(dx, dy, dz) for dz in [-1, 0, 1] for dy in [-1, 0, 1] for dx in [-1, 0, 1]])
assert len(set(DimensionalScanner((-1, 1), (-1, 1), (-1, 1), (-1, 1)).scan())) == 81


In [197]:
from typing import Iterator

class Cube(object):
    def __init__(self, x: int, y: int, z: int = 0, active: bool = False):
        self.x = x
        self.y = y
        self.z = z
        self.active = active

    index_fields = ('x', 'y', 'z')

    @property
    def index(self) -> str:
        return ",".join(map(str, (getattr(self, field) for field in self.index_fields)))

    def neighbours(self, state: "State") -> Iterator[Cube]:
        scanner = DimensionalScanner(*[(-1, 1) for field in self.index_fields])
        self_offset = tuple(0 for field in self.index_fields)

        for offset in scanner.scan():
            if offset != self_offset:
                default_cube = self.__class__(*map(lambda z: z[0] + z[1], zip((getattr(self, field) for field in self.index_fields), offset)))
                yield state.get(default_cube.index, default_cube)

    def active_neighbours(self, state: "State") -> int:
        return sum(1 for neighbour in self.neighbours(state) if neighbour.active)

    def step(self, state: "State") -> Cube:
        active_neighbours = self.active_neighbours(state)

        if self.active:
            return self.with_state(active_neighbours in [2, 3])
        else:
            return self.with_state(active_neighbours == 3)

    def with_state(self, active: bool) -> Cube:
        return self.__class__(*(getattr(self, field) for field in self.index_fields), active=active)

    def __eq__(self, other: Cube) -> bool:
        return self.index == other.index
        
    def __ne__(self, other: Cube) -> bool:
        return self.index != other.index

    def __hash__(self):
        return hash((self.x, self.y, self.z))

## State
The `State` object is the immutable snapshot within which we can evaluate the system. It exposes a `transform()` method which we can use to create a new, derived, `State` with the updated cubes for a given tick.

This class also provides some helpers for printing out 

In [198]:
from typing import Iterable

class State(object):
    def __init__(self, cubes: Iterable[Cube]):
        self.cubes: Dict[str, Cube] = {
            cube.index: cube
            for cube in cubes if cube.active
        }

        self.index_fields = next(iter(self.cubes.values())).index_fields

        self.ranges = tuple(
            (min(getattr(cube, field) for cube in self.cubes.values()), max(getattr(cube, field) for cube in self.cubes.values()))
            for field in self.index_fields
        )

    def transform(self) -> "State":
        new_cube_space = set(neighbour for cube in self.cubes.values() for neighbour in cube.neighbours(self))
        return State(cube.step(self) for cube in new_cube_space)

    def get(self, index: str, default: Cube) -> Cube:
        if index in self.cubes:
            return self.cubes[index]

        return default

    def print(self):
        scanner = DimensionalScanner(*self.ranges[2:])
        for offsets in scanner.scan():
            print(",".join(map(lambda z: f"{z[0]}={z[1]}", zip(self.index_fields[2:], offsets))))
            for y in range(self.ranges[1][0], self.ranges[1][1] + 1):
                print("".join('#' if self.get(",".join(map(str, (x, y, *offsets))), Cube(x, y)).active else '.' for x in range(self.ranges[0][0], self.ranges[0][1] + 1)) + f"  | {self.index_fields[1]} = {y}")

            print("\n")


In [199]:
from typing import Iterable, Dict, Type

class EnergySource(object):
    def __init__(self, init: Iterable[Iterable[str]], cube_type: Type[Cube] = Cube):
        self.state = State(cube_type(x, y, active=(state == '#')) for y, line in enumerate(init) for x, state in enumerate(line))

    def boot(self, print_state: int = 0):
        if print_state > 0:
            print("Before any cycles:\n")
            self.state.print()

        for i in range(0, 6):

            self.step()
            if print_state > i:
                print(f"\nAfter {i + 1} cycle{'s' if i != 0 else ''}")
                self.state.print()

    def step(self):
        self.state = self.state.transform()

    def active_cubes(self) -> int:
        return sum(1 for cube in self.state.cubes.values() if cube.active)

example_source = EnergySource(example)
example_source.boot(print_state=2)
print(f"Active Cubes (Example): {example_source.active_cubes()}")
assert example_source.active_cubes() == 112

source = EnergySource(data)
source.boot()
print(f"Active Cubes (Part 1): {source.active_cubes()}")

Before any cycles:

z=0
.#.  | y = 0
..#  | y = 1
###  | y = 2



After 1 cycle
z=-1
#..  | y = 1
..#  | y = 2
.#.  | y = 3


z=0
#.#  | y = 1
.##  | y = 2
.#.  | y = 3


z=1
#..  | y = 1
..#  | y = 2
.#.  | y = 3



After 2 cycles
z=-2
.....  | y = 0
.....  | y = 1
..#..  | y = 2
.....  | y = 3
.....  | y = 4


z=-1
..#..  | y = 0
.#..#  | y = 1
....#  | y = 2
.#...  | y = 3
.....  | y = 4


z=0
##...  | y = 0
##...  | y = 1
#....  | y = 2
....#  | y = 3
.###.  | y = 4


z=1
..#..  | y = 0
.#..#  | y = 1
....#  | y = 2
.#...  | y = 3
.....  | y = 4


z=2
.....  | y = 0
.....  | y = 1
..#..  | y = 2
.....  | y = 3
.....  | y = 4


Active Cubes (Example): 112
Active Cubes (Part 1): 380


In [201]:
class HyperCube(Cube):
    def __init__(self, x: int, y: int, z: int = 0, w: int = 0, active: bool = False):
        super().__init__(x, y, z, active)
        self.w = w

    index_fields = ('x', 'y', 'z', 'w')

source = EnergySource(data, cube_type=HyperCube)
source.boot()
print(f"Active Cubes (Part 2): {source.active_cubes()}")

Before any cycles:

z=0,w=0
#.#.##.#  | y = 0
#.####.#  | y = 1
...##...  | y = 2
#####.##  | y = 3
#....###  | y = 4
##..##..  | y = 5
#..####.  | y = 6
#...#.#.  | y = 7



After 1 cycle
z=-1,w=-1
...#.....  | y = 0
.........  | y = 1
.#......#  | y = 2
.#.......  | y = 3
#........  | y = 4
#...#...#  | y = 5
#........  | y = 6
....#..#.  | y = 7


z=0,w=-1
...#.....  | y = 0
.........  | y = 1
.#......#  | y = 2
.#.......  | y = 3
#........  | y = 4
#...#...#  | y = 5
#........  | y = 6
....#..#.  | y = 7


z=1,w=-1
...#.....  | y = 0
.........  | y = 1
.#......#  | y = 2
.#.......  | y = 3
#........  | y = 4
#...#...#  | y = 5
#........  | y = 6
....#..#.  | y = 7


z=-1,w=0
...#.....  | y = 0
.........  | y = 1
.#......#  | y = 2
.#.......  | y = 3
#........  | y = 4
#...#...#  | y = 5
#........  | y = 6
....#..#.  | y = 7


z=0,w=0
...#..#..  | y = 0
...#.....  | y = 1
.#......#  | y = 2
.###....#  | y = 3
#.......#  | y = 4
###.#...#  | y = 5
##..#..#.  | y = 6
....##.#.  | y = 